## **<span style="color:lime">Fourth section</span>: *MPC add-ons***
| *Cells* | *Content* |
|---|---|
| 4.1 *Setup* | Imports and helpers |
| 4.2 *Ciphertext aggregation* | Aggregate by token |
| 4.3 *Path serialization* | Stable string format for paths |

#### **<span style="color:purple">Notes</span>**
These helpers are optional and live under `hashcomb.addons`. They are designed to be composed with the core HashComb API.

# HashComb — MPC Add-ons
Minimal utilities for aggregation and serialization workflows.

In [1]:
from pathlib import Path
import sys
import numpy as np

# Make local package importable
root = Path.cwd()

# Walk up to find repository root (pyproject.toml)
repo_root = root
for _ in range(5):
    if (repo_root / "pyproject.toml").exists():
        break
    if repo_root.parent == repo_root:
        break
    repo_root = repo_root.parent

src_dir = repo_root / "src"
if src_dir.exists():
    sys.path.append(str(src_dir))

from hashcomb import Encoder
from hashcomb.addons import aggregate_ciphertexts, serialize_path, deserialize_path

### 4.2 *Ciphertext aggregation*
We group ciphertexts by token and aggregate them without decoding.
This mirrors the paper’s server‑side aggregation step.

In [2]:
enc = Encoder(5, 10.0, 0.0, configPath="artifacts/addons.pkl")
values = [1.2, 1.8, 7.5, 7.7, 7.9]
tokens = [enc.encode(v) for v in values]

ciphertexts = [int(round(v * 10)) for v in values]
items = list(zip(tokens, ciphertexts))

agg = aggregate_ciphertexts(items)
agg

{'268408824': 12, '259612040': 18, '262379111': 152, '192142582': 79}

### 4.3 *Path serialization*
We serialize a hash path into a stable string and round‑trip it back.
This is useful for storage or transport of multihash signatures.

In [3]:
v = 7.5
path = enc.encodePath(v)
serialized = serialize_path(path)
restored = deserialize_path(serialized)

print("Path:", path)
print("Serialized:", serialized)
print("Restored:", restored)
print("Round-trip OK:", path == restored)

Path: ['130576935', '252272603', '237335051', '49137252', '262379111']
Serialized: v1|5|9:130576935|9:252272603|9:237335051|8:49137252|9:262379111
Restored: ['130576935', '252272603', '237335051', '49137252', '262379111']
Round-trip OK: True
